In [4]:
import xarray as xr
import sys
import matplotlib.pyplot as plt
import os
import tqdm
from tqdm import tqdm
import tqdm
import numpy as np
from functools import partial
import json
# changed activation function to hyperbolic tangent
import tensorflow as tf
from tensorflow.keras import layers
AUTOTUNE = tf.data.experimental.AUTOTUNE
from dask.diagnostics import ProgressBar
import pandas as pd
import sys
import os
os.chdir(r'/nesi/project/niwa00018/ML_downscaling_CCAM/A-Robust-Generative-Adversarial-Network-Approach-for-Climate-Downscaling')

# Create an experiment with your api key
sys.path.append(r'/nesi/project/niwa00018/ML_downscaling_CCAM/A-Robust-Generative-Adversarial-Network-Approach-for-Climate-Downscaling')
from src.layers import *
from src.models import *
from src.gan import *
from ops.model_inference.src_eval_inference import *
from tensorflow.keras import layers
import pandas as pd
import json
# changed activation function to hyperbolic tangent
import tensorflow as tf


# TODO LOAD DEFAULT CONFIG VARIABLES
n_members = 10
model_dir = '/nesi/project/niwa00018/ML_downscaling_CCAM/A-Robust-Generative-Adversarial-Network-Approach-for-Climate-Downscaling/models'
filepath = '/nesi/project/niwa00018/ML_downscaling_CCAM/training_GAN/ancil_fields/ERA5_eval_ccam_12km.198110_NZ_Invariant.nc'
output_dir = '/nesi/project/niwa00018/ML_downscaling_CCAM/A-Robust-Generative-Adversarial-Network-Approach-for-Climate-Downscaling/outputs/Reviewer'

# default config_file 
config_file = r'/nesi/project/niwa00018/ML_downscaling_CCAM/A-Robust-Generative-Adversarial-Network-Approach-for-Climate-Downscaling/models/Reviewer_Historical_ACCESS-CM2_0.0001/config_info.json'
with open(config_file) as f:
    config = json.load(f)
historical_period = slice("1985","2014")
future_period = slice("2015","2035")
config["train_x"] ="/nesi/project/niwa00018/ML_downscaling_CCAM/multi-variate-gan/inputs/predictor_fields/predictor_fields_hist_ssp370_merged_updated.nc"
config["train_y"] = "/nesi/project/niwa00018/ML_downscaling_CCAM/multi-variate-gan/inputs/target_fields/target_fields_hist_ssp370_concat.nc"








In [5]:
stacked_X, y, vegt, orog, he = preprocess_input_data(config, match_index =False)

<xarray.DataArray 'orog' ()>
array(1733.36035156) <xarray.DataArray 'he' ()>
array(1157.46264648) <xarray.DataArray 'vegt' ()>
array(17.00000763)


In [6]:

experiments_no_iten = ['Reviewer_Historical_ACCESS-CM2_0.01',
               'Reviewer_Historical_ACCESS-CM2_0.0001',
               'Reviewer_Historical_ACCESS-CM2_0.001',
               'Reviewer_Historical_ACCESS-CM2_0.0025',
               'Reviewer_Historical_ACCESS-CM2_0.005',
               'Reviewer_Historical_ACCESS-CM2_0.01',
               'Reviewer_Historical_ACCESS-CM2_0.1',
               ]

experiments_iten = ['Reviewer_Historical_Intensity_Max_ACCESS-CM2_0.01',
               'Reviewer_Historical_Intensity_Max_ACCESS-CM2_0.0001',
               'Reviewer_Historical_Intensity_Max_ACCESS-CM2_0.001',
               'Reviewer_Historical_Intensity_Max_ACCESS-CM2_0.0025',
               'Reviewer_Historical_Intensity_Max_ACCESS-CM2_0.005',
               'Reviewer_Historical_Intensity_Max_ACCESS-CM2_0.01',
               'Reviewer_Historical_Intensity_Max_ACCESS-CM2_0.1',
               ]



timeslice = slice("1986", "2014")


In [13]:

def run_experiments(experiments, epoch_list, model_dir,
                    input_predictors, common_times, output_shape, orog, he, vegt, n_members, batch_size=64,
                    fixed_unet=True, gcm = None):
    """
    Runs inference on some predictor fields in/ out-of-sample


    experiments: list of experiment names in the model_dir folder
    input_predictors: stacked netcdf with dims (time, lat, lon, channel) and normalized data
    common_times: common_times between output_shape data and input_predictors
    output_shape: a netcdf (y_true) that is the same shape as the output prediction, it contains the time metadata
    orog, he, vegt: auxiliary files from ccam
    n_member: the number of ensemble members

    """

    # if the epoch list is only a float convert to a list
    if isinstance(epoch_list, int):
        epoch_list = [epoch_list] * len(experiments)

    # creating empty lists to save outputs            
    dsets = []
    lambda_var = []
    for i, experiment in enumerate(experiments):
        gan, unet, lambdas = load_model_cascade(experiment, None, model_dir, load_unet=True)
        if i == 0:
            # first instance is always a unet model
            lambdas = 0.0
            preds = xr.concat([predict_parallel_resid(gan, unet,
                                                        input_predictors.sel( GCM =gcm).transpose('time','lat','lon','channel').sel(time = common_times).values,
                                                        output_shape.sel(time=common_times),
                                                        batch_size, orog.values, he.values, vegt.values, model_type='unet')
                                for i in range(n_members)],
                                dim="member")
            
        else:
                # do not change lambdas value otherwise
            lambdas = lambdas
            preds = xr.concat([predict_parallel_resid(gan, unet,
                                                        input_predictors.sel( GCM =gcm).transpose('time','lat','lon','channel').sel(time = common_times).values,
                                                        output_shape.sel(time=common_times),
                                                        batch_size, orog.values, he.values, vegt.values, model_type='GAN')
                                for i in range(n_members)],
                                dim="member")
        lambda_var.append(lambdas)
        dsets.append(preds)
        # finish the experiment and concatenate the data
    dsets = xr.concat(dsets, dim="experiment")
    dsets['experiment'] = (('experiment'), lambda_var)
    dsets = dsets.reindex(experiment=sorted(dsets.experiment.values))
    return dsets

try:
    y = y.isel(GCM =0)[['pr']]
except:
    y =y[['pr']]


In [14]:

for model in list(stacked_X.GCM.values):
    common_time_perfect = stacked_X.sel(time=timeslice).time.to_index().intersection(y.time.to_index())
    output_shape = create_output(stacked_X.sel(time = common_time_perfect), y.sel(time = common_time_perfect))
    output_shape.pr.values = output_shape.pr.values * 0.0
    
    # ensuring that the data is cleared
    # load the imperfect_conditions
    vegt, orog, he = load_and_normalize_topography_data(filepath)

    output_prediction = run_experiments(experiments_no_iten, [], model_dir,
                                        stacked_X, common_time_perfect, output_shape, orog, he, vegt, n_members,
                                        batch_size=64, gcm = model)
    if not os.path.exists(  f'{output_dir}/No_Constraint/'):
        os.makedirs( f'{output_dir}/No_Constraint/')

    if not os.path.exists(  f'{output_dir}/Intensity_Constrained'):
        os.makedirs( f'{output_dir}/Intensity_Constrained')

    output_prediction.to_netcdf(
        f'{output_dir}/No_Constraint/{model}_unconstrained_hist_1986_2005_perfect_framework.nc')


    # output_prediction = run_experiments(experiments_iten, [], model_dir,
    #                                     stacked_X, common_time_perfect, output_shape, orog, he, vegt, n_members,
    #                                     batch_size=64, gcm = model)
    # output_prediction.to_netcdf(
    #     f'{output_dir}/Intensity_Constrained/{model}_IC_hist_1986_2005_perfect_framework.nc')




Max orog: 1733.3603515625, Max he: 1157.462646484375, Max vegt: 17.00000762939453


Predicting:  29%|██▉       | 48/165 [00:35<01:25,  1.37batch/s]


KeyboardInterrupt: 